In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [4]:
import os

#load paths and labels for train and test datasets
path = "/content/drive/MyDrive/scenes/"
folders = os.listdir(path)
image_path1 = []
labels1 = []
for i in folders:
    sub_path = path +i +"/"
    sub_folders = os.listdir(sub_path)
    for j in sub_folders:
        if j[0] == 'P':
            images_folder = os.listdir(sub_path+j)
            for k in images_folder:
                image_path1.append(sub_path+j+'/'+k)
                labels1.append(int(j[11:]))

print(len(image_path1))
print(len(labels1))



path2 = "/content/drive/MyDrive/scenes/test_set/"
folders2 = os.listdir(path2)
image_path2 = []
labels2 = []
for j in folders2:
    if j[0] == 'P':
        images_folder = os.listdir(path2+j)
        for k in images_folder:
            image_path2.append(path2+j+'/'+k)
            labels2.append(int(j[11:]))

print(len(image_path2))
print(len(labels2))

1335
1335
225
225


In [1]:
from __future__ import print_function 
from __future__ import division
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.10.0
Torchvision Version:  0.11.1


In [4]:
#initialize the variables
data_dir = "/content/drive/MyDrive/scenes"
num_classes = 15
batch_size = 15
num_epochs = 18
feature_extract = False

In [7]:
#train function
def train_model(model, dataloaders, criterion1, criterion2, optimizer, num_epochs):
    since = time.time()
    val_acc_history = []
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                  #outputs = model(inputs)
                  outputs, layer1, layer2 = model.forward(inputs)
                  v1 = torch.subtract(layer1,torch.mean(layer1))
                  v2 = torch.subtract(layer2,torch.mean(layer2))
                  loss0 = criterion1(outputs, labels)
                  loss1 = criterion2(layer1, v1)
                  loss2 = criterion2(layer2, v2)
                  loss = 0.5*loss0 + 0.25*loss1 + 0.25*loss2

                  _, preds = torch.max(outputs, 1)

                  # backward + optimize only if in training phase
                  if phase == 'train':
                    loss.backward()
                    optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [8]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ResNet18(nn.Module):
    def __init__(self, orig_model):
        super(ResNet18, self).__init__()
        self.orig = nn.Sequential(*(list(orig_model.children())[:-1])).to(device)
        for param in self.orig.parameters():
            param.requires_grad = True
            # Replace the last fully-connected layer
            # Parameters of newly constructed modules have requires_grad=True by default
        self.fc = nn.Linear(512, 256).to(device)
        self.fc2 = nn.Linear(256, 128).to(device)
        self.fc3 = nn.Linear(128, 15).to(device)
        #self.fc4 = nn.Linear(64, 15).to(device)
        #self.fc5 = nn.Linear(32, 15).to(device)

    def forward(self, x):
        x = self.orig(x)
        x = self.bn(x)
        x = x.view(x.size(0), -1)
        x1 = F.relu(self.bn2(self.fc(x)))
        x = self.drop(x1)
        x2 = F.relu(self.bn3(self.fc2(x)))
        x = self.drop(x2)
        #x3 = F.relu(self.bn4(self.fc3(x)))
        #x = self.drop(x3)
        #x4 = F.relu(self.bn5(self.fc4(x)))
        #x = self.drop(x4)
        x = F.relu(self.fc3(x))
        p = F.softmax(x, dim=1)
        return x, p, x1, x2


class AlexNet(nn.Module):

    def __init__(self, orig_model):
        
        super(AlexNet, self).__init__()
        #self.b0 = nn.BatchNorm2d(6400).to(device)
        #self.b1 = nn.BatchNorm2d(4096)
        #self.b2 = nn.BatchNorm2d(4096)
    
        self.orig = nn.Sequential(*(list(orig_model.children())[:-1])).to(device)
        for param in self.orig.parameters():
            param.requires_grad = True
        self.c1 = nn.Linear(256 * 6 * 6, 4096) #0
        self.c2 =  nn.ReLU(inplace=True)  # 1
        self.c3 = nn.Dropout(0.5)  # 2
        self.c4 = nn.Linear(4096, 4096)  # 3
        self.c5 = nn.ReLU(inplace=True)  # 4
        self.c6 = nn.Dropout(0.5)  # 5
        self.c7 = nn.Linear(4096, num_classes)  # 6
    

    def forward(self, x):
        x = self.orig(x)
        #x = self.b0(x)
        x = x.view(x.size(0), 256 * 6 * 6)
        x1 = self.c1(x)
        x2 = self.c2(x1)
        x3 = self.c3(x2)
        x4 = self.c4(x3)
        x5 = self.c5(x4)
        x6 = self.c6(x5)
        x = self.c7(x6)
        return x, x1, x4




input_size = 224

# pretrained_model = torchvision.models.resnet18(pretrained=True)
model_p = torchvision.models.alexnet(pretrained=True)
# model_ft = ResNet18(pretrained_model)
model_ft = AlexNet(model_p)
input_size = 224

# Print the model we just instantiated
print(model_ft)

AlexNet(
  (orig): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
      (1): ReLU(inplace=True)
      (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (4): ReLU(inplace=True)
      (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
      (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (7): ReLU(inplace=True)
      (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (9): ReLU(inplace=True)
      (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (1): AdaptiveAvgPool2d(output_size=(6, 6))
  )
  (c1): Linear(in_features=9216, out_features=4096, bias=True)
  (c2): ReLU(inplace=Tru

In [10]:
#preprocess the data
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, name),
                                          data_transforms[x])
                  for x, name in [['train', "train_set"], ['val', "test_set"]]}


dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size = 8, shuffle=True)
              for x in ['train', 'val']}

Initializing Datasets and Dataloaders...


In [6]:
# Send the model to GPU
model_ft = model_ft.to(device)

params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 orig.0.0.weight
	 orig.0.0.bias
	 orig.0.3.weight
	 orig.0.3.bias
	 orig.0.6.weight
	 orig.0.6.bias
	 orig.0.8.weight
	 orig.0.8.bias
	 orig.0.10.weight
	 orig.0.10.bias
	 c1.weight
	 c1.bias
	 c4.weight
	 c4.bias
	 c7.weight
	 c7.bias


In [14]:
# Setup the loss fxn
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.MSELoss()


# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion1, criterion2, optimizer_ft, num_epochs=num_epochs)


Epoch 0/17
----------
train Loss: 0.3870 Acc: 0.8000
val Loss: 0.0904 Acc: 1.0000

Epoch 1/17
----------
train Loss: 0.3171 Acc: 0.8207
val Loss: 0.0674 Acc: 1.0000

Epoch 2/17
----------
train Loss: 0.2826 Acc: 0.8351
val Loss: 0.0581 Acc: 1.0000

Epoch 3/17
----------
train Loss: 0.2378 Acc: 0.8550
val Loss: 0.0505 Acc: 1.0000

Epoch 4/17
----------
train Loss: 0.2381 Acc: 0.8577
val Loss: 0.0429 Acc: 1.0000

Epoch 5/17
----------
train Loss: 0.2292 Acc: 0.8550
val Loss: 0.0383 Acc: 1.0000

Epoch 6/17
----------
train Loss: 0.2103 Acc: 0.8757
val Loss: 0.0342 Acc: 1.0000

Epoch 7/17
----------
train Loss: 0.1911 Acc: 0.8928
val Loss: 0.0357 Acc: 1.0000

Epoch 8/17
----------
train Loss: 0.1925 Acc: 0.8820
val Loss: 0.0269 Acc: 1.0000

Epoch 9/17
----------
train Loss: 0.1719 Acc: 0.9018
val Loss: 0.0271 Acc: 1.0000

Epoch 10/17
----------
train Loss: 0.1929 Acc: 0.8784
val Loss: 0.0261 Acc: 1.0000

Epoch 11/17
----------
train Loss: 0.1808 Acc: 0.8784
val Loss: 0.0262 Acc: 1.0000

Ep

AttributeError: ignored

In [17]:
torch.save(model_ft.state_dict(), "/content/drive/MyDrive/saved_model/model")